Import Libraries and Load Dataset

In [5]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# Set the path to the dataset
dataset_path = 'mnist_middle_columns'

# Load the images and their corresponding labels
images = []
labels = []

for i in range(60000):
    # Load the image
    img_path = os.path.join(dataset_path, '{}.jpg'.format(i))
    img = Image.open(img_path).convert('L')  # Convert to grayscale
    img = np.array(img)
    
    # Load the label
    label_path = os.path.join(dataset_path, '{}.csv'.format(i))
    label = np.loadtxt(label_path, delimiter=',')
    label = label[1:]
    
    # Set the middle column of pixels to zero
    img[:, 14] = 0 
    
    # Convert to tensors and append to the lists
    images.append(torch.from_numpy(img))
    labels.append(torch.from_numpy(label))

Split Dataset and Create Custom Dataset and Dataloader

In [6]:
# Split the data into train and test sets
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create custom dataset and dataloader
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx].float(), self.labels[idx].float()


train_dataset = MyDataset(images_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = MyDataset(images_test, labels_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Define the Model (MLP) and Instantiate

In [8]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 28)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x.float()

# Instantiate the MLP model and define the loss function and optimizer
model = MLP()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train the Model

In [11]:
# Train the MLP model
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        labels = labels.view(-1, 28)  # reshape labels to match outputs
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Print the average loss for the epoch
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader)))

    # Test the MLP model
    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            labels = labels.view(-1, 28)  # reshape labels to match outputs
            test_loss += criterion(outputs, labels).item()

        # Print the test loss
        print('Test Loss: {:.4f}'.format(test_loss/len(test_loader)))

Epoch [1/20], Train Loss: 0.0155
Test Loss: 0.0163
Epoch [2/20], Train Loss: 0.0155
Test Loss: 0.0166
Epoch [3/20], Train Loss: 0.0154
Test Loss: 0.0162
Epoch [4/20], Train Loss: 0.0154
Test Loss: 0.0163
Epoch [5/20], Train Loss: 0.0154
Test Loss: 0.0163
Epoch [6/20], Train Loss: 0.0154
Test Loss: 0.0167
Epoch [7/20], Train Loss: 0.0154
Test Loss: 0.0164
Epoch [8/20], Train Loss: 0.0154
Test Loss: 0.0160
Epoch [9/20], Train Loss: 0.0155
Test Loss: 0.0168
Epoch [10/20], Train Loss: 0.0153
Test Loss: 0.0170
Epoch [11/20], Train Loss: 0.0154
Test Loss: 0.0171
Epoch [12/20], Train Loss: 0.0154
Test Loss: 0.0177
Epoch [13/20], Train Loss: 0.0154
Test Loss: 0.0167
Epoch [14/20], Train Loss: 0.0154
Test Loss: 0.0165
Epoch [15/20], Train Loss: 0.0154
Test Loss: 0.0161
Epoch [16/20], Train Loss: 0.0153
Test Loss: 0.0168
Epoch [17/20], Train Loss: 0.0153
Test Loss: 0.0166
Epoch [18/20], Train Loss: 0.0153
Test Loss: 0.0166
Epoch [19/20], Train Loss: 0.0153
Test Loss: 0.0175
Epoch [20/20], Train 